# Test the EKF_Net backpropogation

In [495]:
import time, os, json
import numpy as np
import matplotlib.pyplot as plt

In [496]:
from cs231n.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from EKFNet import EKFNet_layers 

from EKFNet import EKFNet 
from loadData import raw

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [497]:
## Load the KITTI data
# Change this to the directory where you store KITTI data
basedir = '../data_test/'

# Specify the dataset to load
date = '2011_09_26'
drive = '0022'

# Load the data. Optionally, specify the frame range to load.
# dataset = pykitti.raw(basedir, date, drive)
dataset      = raw(basedir, date, drive, frames=range(7800, 8100, 10))
#dataset      = raw(basedir, date, drive)
#print("Selected File Names : ")
#for item in dataset_full.oxts_files:
#    print(item)
print("Total Lenght of the time data ", len(dataset))

Total Lenght of the time data  30


## Forward Steps

In [501]:
EKF_filter = EKFNet.EKFNet()
EKF_filter.load_data_set(dataset)
#EKF_filter.add_some_noise()
EKF_filter.run_kf()

Running KF Liang XU


In [503]:
EKF_filter.plot_overview(update_rule='adam')

TypeError: plot_overview() got an unexpected keyword argument 'update_rule'

## Backward Steps
In here I am going to test the back ward steps for each equations
### Update Equations
#### function 9
x_post = x_hat + np.dot(K, y_)

In [7]:
def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [7]:
#np.random.seed(232)
# 3 inputs
x_hat   = np.random.randn(5,1)
K       = np.random.randn(5,3)
y_      = np.random.randn(3,1)
out, cache = EKFNet_layers.fun_update_9_forward(x_hat, K, y_)

dx_post = np.random.randn(*out.shape)
fx_hat = lambda x_hat: EKFNet_layers.fun_update_9_forward(x_hat, K, y_)[0]
fK     = lambda K    : EKFNet_layers.fun_update_9_forward(x_hat, K, y_)[0]
fy_    = lambda y_   : EKFNet_layers.fun_update_9_forward(x_hat, K, y_)[0]
dx_hat_num = eval_numerical_gradient_array(fx_hat, x_hat, dx_post)
dK_num     = eval_numerical_gradient_array(fK,     K,     dx_post)
dy_num     = eval_numerical_gradient_array(fy_,    y_,    dx_post)

# run real backward

dx_hat, dK, dy_ = EKFNet_layers.fun_update_9_back(dx_post, x_hat, K, y_)
print('dx_hat error: ', rel_error(dx_hat_num, dx_hat))
print('dK     error: ', rel_error(dK_num,     dK))
print('dy_num      : ', rel_error(dy_num, dy_))
print(dx_hat)
print(dx_hat_num)

dx_hat error:  3.2755995793143366e-12
dK     error:  2.0608070170527488e-11
dy_num      :  1.6306273868402972e-12
[[-1.6940719 ]
 [-0.01202285]
 [ 0.85915533]
 [-0.28692244]
 [ 0.95975205]]
[[-1.6940719 ]
 [-0.01202285]
 [ 0.85915533]
 [-0.28692244]
 [ 0.95975205]]


#### Function 8

In [8]:
K   = np.random.randn(5,3)
H   = np.random.randn(3,5)
P   = np.random.randn(5,5)
out, cache = EKFNet_layers.fun_update_8_forward(P, K, H)
dp_post = np.random.randn(*out.shape)

fK  = lambda K: EKFNet_layers.fun_update_8_forward(P, K, H)[0]
fH  = lambda H: EKFNet_layers.fun_update_8_forward(P, K, H)[0]
fP  = lambda P: EKFNet_layers.fun_update_8_forward(P, K, H)[0]

dK_num = eval_numerical_gradient_array(fK, K, dp_post)
dH_num = eval_numerical_gradient_array(fH, H, dp_post)
dP_num = eval_numerical_gradient_array(fP, P, dp_post)

dp_hat, dK, dH = EKFNet_layers.fun_update_8_back(dp_post, P, K, H)

print('dp_hat error: ', rel_error(dP_num, dp_hat))
print('dK error: '    , rel_error(dK_num, dK))
print('dH error: '    , rel_error(dH_num, dH))


fun_8 back 
dp_hat error:  8.435478421833332e-11
dK error:  4.326563231671625e-11
dH error:  9.295691903194977e-11


#### Function 7

In [9]:
S   = np.random.randn(3,3)
H   = np.random.randn(3,5)
P   = np.random.randn(5,5)
out, cache = EKFNet_layers.fun_update_7_forward(P, H, S)
dk_post = np.random.randn(*out.shape)

fS  = lambda S: EKFNet_layers.fun_update_7_forward(P, H, S)[0]
fH  = lambda H: EKFNet_layers.fun_update_7_forward(P, H, S)[0]
fP  = lambda P: EKFNet_layers.fun_update_7_forward(P, H, S)[0]

dS_num = eval_numerical_gradient_array(fS, S, dk_post)
dH_num = eval_numerical_gradient_array(fH, H, dk_post)
dP_num = eval_numerical_gradient_array(fP, P, dk_post)

dP_hat, dH, dS = EKFNet_layers.fun_update_7_back(dk_post, P, H, S)
print(dH)
print(dH_num)
print('dp_hat error: ', rel_error(dP_num, dP_hat))
print('dS error: '    , rel_error(dS_num, dS))
print('dH error: '    , rel_error(dH_num, dH))

[[-3.96860751  4.46770918  1.23944146  4.70098056  0.60063016]
 [-2.77050057  4.3622378   0.92425106  3.148116   -1.38320923]
 [-2.1506515   4.5427981   2.17638157  0.95691682 -4.01209954]]
[[-3.96860751  4.46770918  1.23944146  4.70098056  0.60063016]
 [-2.77050057  4.3622378   0.92425106  3.148116   -1.38320923]
 [-2.1506515   4.54279811  2.17638157  0.95691682 -4.01209954]]
dp_hat error:  5.038387698117703e-10
dS error:  2.6347520264237358e-11
dH error:  2.207296652488631e-11


#### Inverse

In [10]:
#np.random.seed(232)
np.set_printoptions(precision=4)
S   = np.random.randn(2,2)
#S[0][0] = 1
#S[1][1] = 2
#S[0][1] = 0
#S[1][0] = 0

print("S is \n", S)
a = S[0][0]
b = S[0][1]
c = S[1][0]
d = S[1][1]
base = (a * d - b * c)**2
ele1 = -d**2 / base
ele2 = b * d / base
ele3 = c* d / base
ele4 = -b * c / base
print('%.4f'%ele1)
print('%.4f'%ele2)
print('%.4f'%ele3)
print('%.4f'%ele4)
print()
ele1 = -d**2 / base
ele2 = c * d / base
ele3 = b * d / base
ele4 = -b * c / base
print('%.4f'%ele1)
print('%.4f'%ele2)
print('%.4f'%ele3)
print('%.4f'%ele4)

out = EKFNet_layers.inverse_dev_forward(S)
dS_inv = np.random.randn(*out.shape)
#dS_inv[0][0] = 1
#dS_inv[0][1] = 1
#dS_inv[1][0] = 1
#dS_inv[1][1] = 1
fS  = lambda S: EKFNet_layers.inverse_dev_forward(S)[0]

dS_num = eval_numerical_gradient_array(fS, S, dS_inv, h = 0.0000001)
dS = EKFNet_layers.inverse_dev_back(dS_inv, S)
print('dS error: '    , rel_error(dS_num, dS))

print("\n ds_num error")
print(dS_num)
print("My error")
print(dS)

S is 
 [[-0.3212 -1.3297]
 [ 1.763   0.6747]]
-0.1006
-0.1982
0.2628
0.5179

-0.1006
0.2628
-0.1982
0.5179
[[-0.1006 -0.1982  0.2628  0.5179]
 [ 0.2628  0.0479 -0.6867 -0.1251]
 [-0.1982 -0.3906  0.0479  0.0944]
 [ 0.5179  0.0944 -0.1251 -0.0228]]
[[0.7762 0.0378]
 [0.8468 3.0623]]
[[1.6231 3.1002 0.     0.    ]]
dS error:  1.3480026580842929e-09

 ds_num error
[[-0.7777  0.575 ]
 [-1.5326  1.1331]]
My error
[[-0.7777  0.575 ]
 [-1.5326  1.1331]]


In [11]:
#np.random.seed(232)
np.set_printoptions(precision=4)
S   = np.random.randn(3,3)

out = EKFNet_layers.inverse_dev_forward(S)
dS_inv = np.random.randn(*out.shape)
#dS_inv[0][0] = 1
#dS_inv[0][1] = 0
#dS_inv[0][2] = 0
#dS_inv[1][0] = 0
#dS_inv[1][1] = 0
#dS_inv[1][2] = 0
#dS_inv[2][0] = 0
#dS_inv[2][1] = 1
#dS_inv[2][2] = 0
fS  = lambda S: EKFNet_layers.inverse_dev_forward(S)[0]

dS_num = eval_numerical_gradient_array(fS, S, dS_inv, h = 0.0000001)
dS = EKFNet_layers.inverse_dev_back_3(dS_inv, S)
print('dS error: '    , rel_error(dS_num, dS))

print("\n ds_num error")
print(dS_num)
print("My error")
print(dS)

dS error:  1.0

 ds_num error
[[ 0.0528 -0.003   0.098 ]
 [-0.372   0.021  -0.6896]
 [-0.1871  0.0105 -0.3468]]
My error
[[-0.1059 -0.1863  0.3251]
 [-0.7816 -0.4908 -1.1362]
 [ 0.2329  0.5227 -0.2257]]


#### Function 6

In [73]:
R   = np.random.randn(3,3)
H   = np.random.randn(3,5)
P   = np.random.randn(5,5)

out, cache = EKFNet_layers.fun_update_6_forward(H, P, R)
dS = np.random.randn(*out.shape)

fR  = lambda R: EKFNet_layers.fun_update_6_forward(H, P, R)[0]
fH  = lambda H: EKFNet_layers.fun_update_6_forward(H, P, R)[0]
fP  = lambda P: EKFNet_layers.fun_update_6_forward(H, P, R)[0]

dR_num = eval_numerical_gradient_array(fR, R, dS)
dH_num = eval_numerical_gradient_array(fH, H, dS)
dP_num = eval_numerical_gradient_array(fP, P, dS)

dH, dP, dR = EKFNet_layers.fun_update_6_back(dS,H,P, R)

print('dp_hat error: ', rel_error(dP_num, dP))
print('dR error: '    , rel_error(dR_num, dR))
print('dH error: '    , rel_error(dH_num, dH))
print(dR)
print(dS)
print(dR_num)


dp_hat error:  1.0682495577824146e-10
dR error:  2.5480091403192562e-11
dH error:  1.8855618120259943e-09
[[-0.9227 -0.7611  0.0245]
 [-1.9505  0.2698  0.8998]
 [-1.3565 -0.3235 -1.0141]]
[[-0.9227 -0.7611  0.0245]
 [-1.9505  0.2698  0.8998]
 [-1.3565 -0.3235 -1.0141]]
[[-0.9227 -0.7611  0.0245]
 [-1.9505  0.2698  0.8998]
 [-1.3565 -0.3235 -1.0141]]


#### Function 5

In [72]:
z = np.random.randn(3)
x_hat = np.random.randn(5)
gen_h = EKF_filter.gen_h
gen_H = EKF_filter.gen_H
out, cache = EKFNet_layers.fun_update_5_forward(z, x_hat ,gen_h, gen_H)
dy  = np.random.randn(*out.shape)
fx  = lambda x_hat: EKFNet_layers.fun_update_5_forward(z, x_hat ,gen_h, gen_H)[0]
dx_num  = eval_numerical_gradient_array(fx, x_hat, dy)
dx = EKFNet_layers.fun_update_5_back(dy, z, x_hat ,gen_h, gen_H)

print('dy error: ', rel_error(dx_num, dx))
print(dx_num)
print(dx)

[[ 1.      0.     -0.3228  0.      0.    ]
 [ 0.      1.     -0.026   0.      0.    ]
 [ 0.      0.      1.      0.      0.    ]]
dy error:  5.3473905530394814e-12
[0.6414 0.6401 1.2825 0.     0.    ]
[[ 0.6414  0.6401  1.2825 -0.     -0.    ]]


### Prediction step

#### Fucntion 2
P = FPFT + Q

In [119]:
x_post = np.random.randn(5,1)
p_post = np.random.randn(5,5)
dt = np.random.rand()
f_gen = EKF_filter.G
wheelbase = EKF_filter.h
Q = np.random.randn(5,5)
out, cache = EKFNet_layers.fun_predict_2_forward(Q, x_post, p_post, dt ,f_gen)
dp_pred  = np.random.randn(*out.shape)
fx_post  = lambda x_post: EKFNet_layers.fun_predict_2_forward(Q, x_post, p_post, dt ,f_gen)[0]
fp_post  = lambda p_post: EKFNet_layers.fun_predict_2_forward(Q, x_post, p_post, dt ,f_gen)[0]
fq       = lambda Q     : EKFNet_layers.fun_predict_2_forward(Q, x_post, p_post, dt ,f_gen)[0]

dx_post_num  = eval_numerical_gradient_array(fx_post, x_post, dp_pred)
dp_post_num  = eval_numerical_gradient_array(fp_post, p_post, dp_pred)
dq_num       = eval_numerical_gradient_array(fq     , Q     , dp_pred, h = 10)


dx_post, dp_post, dq = EKFNet_layers.fun_predict_2_back(dp_pred, x_post, p_post, dt ,f_gen, Q, wheelbase)

print(dx_post_num)
print(dx_post)
print("dq error: ", rel_error(dq_num, dq))
print("dp_post error: ", rel_error(dp_post_num, dp_post))
print("dx_post_error: ", rel_error(dx_post_num, dx_post))

[[-0.8734  1.1224  2.9441  0.5182  2.3892]
 [-1.206  -2.3725  1.9797 -0.4352  0.4311]
 [-2.9239 -1.6812 -1.6115 -5.6653 -0.4995]
 [-0.2868  6.088   0.1598 -6.2743  2.4583]
 [ 0.8315 -2.6658  1.719   2.7324 -2.8466]]
[[ 0.    ]
 [ 0.    ]
 [-0.0865]
 [-0.0181]
 [-0.1029]]
[[ 0.    ]
 [ 0.    ]
 [-0.0865]
 [-0.0181]
 [-0.1029]]
dq error:  6.912769769492817e-17
dp_post error:  5.336152539949314e-11
dx_post_error:  5.089868665635711e-10


In [51]:
dt = np.random.rand()
dt

0.07477958271937835

In [210]:
x_post = np.random.randn(5,1)

print(x_post)

f_gen = EKF_filter.G
f_pred = EKF_filter.predict
out, cache = EKFNet_layers.fun_predict_1_forward(x_post, f_pred, dt)

dx_pred  = np.random.randn(*out.shape)

fx_post  = lambda x_post: EKFNet_layers.fun_predict_1_forward(x_post, f_pred, dt)[0]
dx_post_num  = eval_numerical_gradient_array(fx_post, x_post, dx_pred)

dx_post  = EKFNet_layers.fun_predict_1_back(dx_pred, x_post, f_pred, dt, f_gen)
print(dx_post)
print(dx_post_num)
print("dx_post error: ", rel_error(dx_post, dx_post_num))

[[-0.1014]
 [-1.1319]
 [-0.8544]
 [-1.5198]
 [-1.1795]]
[[-0.8896]
 [ 1.5312]
 [-0.0978]
 [ 0.8042]
 [ 1.1779]]
[[-0.8896]
 [ 1.5312]
 [-0.0978]
 [ 0.8042]
 [ 1.1779]]
dx_post error:  2.3455160590487334e-06


### S update respect to x

In [231]:
R   = np.random.randn(3,3)
x   = np.random.randn(5)
P   = np.random.randn(5,5)
gen_H = EKF_filter.gen_H
out, cache = EKFNet_layers.fun_update_6_2_forward(x, P, R, gen_H)
dS = np.random.randn(*out.shape)

fR  = lambda R: EKFNet_layers.fun_update_6_2_forward(x, P, R, gen_H)[0]
fx  = lambda x: EKFNet_layers.fun_update_6_2_forward(x, P, R, gen_H)[0]
fP  = lambda P: EKFNet_layers.fun_update_6_2_forward(x, P, R, gen_H)[0]

dR_num = eval_numerical_gradient_array(fR, R, dS)
dx_num = eval_numerical_gradient_array(fx, x, dS)
dP_num = eval_numerical_gradient_array(fP, P, dS)

dx, dP, dR = EKFNet_layers.fun_update_6_2_back(dS,x, P, R, gen_H, EKF_filter)

print('dp_hat error: ', rel_error(dP_num, dP))
print('dR error: '    , rel_error(dR_num, dR))
print('dx error: '    , rel_error(dx_num, dx))
print(dx)
print(dx_num)

dp_hat error:  3.1721234880040185e-06
dR error:  4.1370183828729235e-08
dx error:  4.555855629274916e-06
[0.     0.     0.1973 0.     0.    ]
[0.     0.     0.1972 0.     0.    ]


## Update Forward Check

In [286]:
#update_forward(self, x_pred, p_pred, H, S, y_)
x_pred = np.random.randn(5,1)
p_pred = np.random.randn(5,5)
H      = np.random.randn(3,5)
S      = np.random.randn(3,3)
y_     = np.random.randn(3,1)
x_post, p_post, cache = EKFNet_layers.update_forward(x_pred, p_pred, H, S, y_)

# Check the deriative respect to dx_post
dx_post =   np.random.randn(*x_post.shape)
dp_post =   np.zeros(p_post.shape)
# Numerical Results
fx_pred  = lambda x_pred: EKFNet_layers.update_forward(x_pred, p_pred, H, S, y_)[0]
fp_pred  = lambda p_pred: EKFNet_layers.update_forward(x_pred, p_pred, H, S, y_)[0]
fH  = lambda H: EKFNet_layers.update_forward(x_pred, p_pred, H, S, y_)[0]
fS  = lambda S: EKFNet_layers.update_forward(x_pred, p_pred, H, S, y_)[0]
fy_  = lambda y_: EKFNet_layers.update_forward(x_pred, p_pred, H, S, y_)[0]

dx_pred_num  = eval_numerical_gradient_array(fx_pred, x_pred, dx_post)
dp_pred_num  = eval_numerical_gradient_array(fp_pred, p_pred, dx_post)
dH_num       = eval_numerical_gradient_array(fH, H, dx_post)
dS_num       = eval_numerical_gradient_array(fS, S, dx_post)
dy_num       = eval_numerical_gradient_array(fy_, y_, dx_post)

dx_pred, dp_pred, dH, dS, dy_ = EKFNet_layers.update_backward(dx_post, dp_post, cache)

print('dx_pred error: ', rel_error(dx_pred_num, dx_pred))
print('dp_pred error: ', rel_error(dp_pred_num, dp_pred))
print('dH      error: ', rel_error(dH_num, dH))
print('dS      error: ', rel_error(dS_num, dS))
print('dy_     error: ', rel_error(dy_num, dy_))


dx_pred error:  3.2756416086162146e-12
dp_pred error:  2.904422160549565e-10
dH      error:  9.381979382305789e-09
dS      error:  7.399579111565025e-10
dy_     error:  3.0418397380144948e-12


In [298]:
#update_forward(self, x_pred, p_pred, H, S, y_)
x_pred = np.random.randn(5,1)
p_pred = np.random.randn(5,5)
H      = np.random.randn(3,5)
S      = np.random.randn(3,3)
y_     = np.random.randn(3,1)
x_post, p_post, cache = EKFNet_layers.update_forward(x_pred, p_pred, H, S, y_)
# Check the deriative respect to dp_post
dx_post =   np.zeros(x_post.shape)
dp_post =   np.random.randn(*p_post.shape)
# Numerical Results
fx_pred  = lambda x_pred: EKFNet_layers.update_forward(x_pred, p_pred, H, S, y_)[1]
fp_pred  = lambda p_pred: EKFNet_layers.update_forward(x_pred, p_pred, H, S, y_)[1]
fH  = lambda H: EKFNet_layers.update_forward(x_pred, p_pred, H, S, y_)[1]
fS  = lambda S: EKFNet_layers.update_forward(x_pred, p_pred, H, S, y_)[1]
fy_  = lambda y_: EKFNet_layers.update_forward(x_pred, p_pred, H, S, y_)[1]

dx_pred_num  = eval_numerical_gradient_array(fx_pred, x_pred, dp_post)
dp_pred_num  = eval_numerical_gradient_array(fp_pred, p_pred, dp_post)
dH_num       = eval_numerical_gradient_array(fH, H, dp_post)
dS_num       = eval_numerical_gradient_array(fS, S, dp_post)
dy_num       = eval_numerical_gradient_array(fy_, y_, dp_post)

dx_pred, dp_pred, dH, dS, dy_ = EKFNet_layers.update_backward(dx_post, dp_post, cache)

print('dx_pred error: ', rel_error(dx_pred_num, dx_pred))
print('dp_pred error: ', rel_error(dp_pred_num, dp_pred))
print('dH      error: ', rel_error(dH_num, dH))
print('dS      error: ', rel_error(dS_num, dS))
print('dy_     error: ', rel_error(dy_num, dy_))

dx_pred error:  0.0
dp_pred error:  1.3908042815199196e-09
dH      error:  1.8684469139319994e-10
dS      error:  2.374438576104069e-09
dy_     error:  0.0


## Measurement Check Up

In [300]:
EKF_filter = EKFNet.EKFNet()
EKF_filter.load_data_set(dataset)

In [339]:
#EKF_filter.imu_off_l
EKF_filter.imu_alpha

1.4157995848709557

In [386]:
np.zeros((5,), dtype=np.float64)
R = np.zeros((5,5), dtype=np.float64)
"""
def measurement_forward(x_pred, 
                        p_pred, 
                        z, 
                        sigma_GPS_x, 
                        sigma_GPS_y,
                        sigma_GPS_h,
                        gen_H,
                        gen_h):
"""

x_pred = np.random.randn(5)
p_pred = np.random.randn(5,5)
z      = np.random.randn(3)
R      = np.random.randn(3,3)

gen_H = EKF_filter.gen_H
gen_h = EKF_filter.gen_h
y_, S, H, cache = EKFNet_layers.measurement_forward(x_pred, p_pred, z, R ,gen_H, gen_h)

dy_ =   np.zeros(y_.shape)
dS  =   np.random.randn(*S.shape)
dH  =   np.random.randn(*H.shape)
dH  =   np.zeros(H.shape)

#fx_pred  = lambda x_pred: EKFNet_layers.update_forward(x_pred, p_pred, z, sigma_GPS_x, 
#                                                    sigma_GPS_y,sigma_GPS_h,gen_H, gen_h)[0]
fR  = lambda R : EKFNet_layers.measurement_forward(x_pred, p_pred, z, R ,gen_H, gen_h)[1]
fp_pred  = lambda p_pred: EKFNet_layers.measurement_forward(x_pred, p_pred, z, R ,gen_H, gen_h)[1]
fx_pred  = lambda x_pred: EKFNet_layers.measurement_forward(x_pred, p_pred, z, R ,gen_H, gen_h)[1]

dR_num  = eval_numerical_gradient_array(fR, R, dS)
dp_pred_num =  eval_numerical_gradient_array(fp_pred, p_pred, dS)
dx_pred_num = eval_numerical_gradient_array(fx_pred, x_pred, dS)

dx_pred, dp_pred, dR = EKFNet_layers.measurement_backward(dy_, dS, dH, cache)

print(dx_pred_num)
print(dx_pred)
print('dx_pred error: ', rel_error(dx_pred_num, dx_pred))
print('dp_pred error: ', rel_error(dp_pred_num, dp_pred))
print("dR      error: ", rel_error(dR_num, dR))

[0.     0.     1.2059 0.     0.    ]
[[0.     0.     1.2059 0.     0.    ]]
dx_pred error:  4.141069107639035e-12
dp_pred error:  4.422801616768121e-12
dR      error:  3.2755994679541575e-12


In [385]:
np.zeros((5,), dtype=np.float64)
R = np.zeros((5,5), dtype=np.float64)
"""
def measurement_forward(x_pred, 
                        p_pred, 
                        z, 
                        sigma_GPS_x, 
                        sigma_GPS_y,
                        sigma_GPS_h,
                        gen_H,
                        gen_h):
"""

x_pred = np.random.randn(5)
p_pred = np.random.randn(5,5)
z      = np.random.randn(3)
R      = np.random.randn(3,3)

gen_H = EKF_filter.gen_H
gen_h = EKF_filter.gen_h
y_, S, H, cache = EKFNet_layers.measurement_forward(x_pred, p_pred, z, R ,gen_H, gen_h)

dy_ =   np.random.randn(*y_.shape)
dS  =   np.zeros(S.shape)
dH  =   np.random.randn(*H.shape)
dH  =   np.zeros(H.shape)

#fx_pred  = lambda x_pred: EKFNet_layers.update_forward(x_pred, p_pred, z, sigma_GPS_x, 
#                                                    sigma_GPS_y,sigma_GPS_h,gen_H, gen_h)[0]
fR  = lambda R : EKFNet_layers.measurement_forward(x_pred, p_pred, z, R ,gen_H, gen_h)[0]
fp_pred  = lambda p_pred: EKFNet_layers.measurement_forward(x_pred, p_pred, z, R ,gen_H, gen_h)[0]
fx_pred  = lambda x_pred: EKFNet_layers.measurement_forward(x_pred, p_pred, z, R ,gen_H, gen_h)[0]

dR_num  = eval_numerical_gradient_array(fR, R, dy_)
dp_pred_num =  eval_numerical_gradient_array(fp_pred, p_pred, dy_)
dx_pred_num = eval_numerical_gradient_array(fx_pred, x_pred, dy_)

dx_pred, dp_pred, dR = EKFNet_layers.measurement_backward(dy_, dS, dH, cache)

print(dx_pred_num)
print(dx_pred)
print('dx_pred error: ', rel_error(dx_pred_num, dx_pred))
print('dp_pred error: ', rel_error(dp_pred_num, dp_pred))
print("dR      error: ", rel_error(dR_num, dR))


[-1.6966 -1.3708 -0.8193  0.      0.    ]
[[-1.6966 -1.3708 -0.8193  0.      0.    ]]
dx_pred error:  5.620721533557004e-12
dp_pred error:  0.0
dR      error:  0.0


## Working on the Prediction Forward
### Check the input is dp_pred

In [489]:
"""
def prediction_forward(x_post, p_post, dt, 
                       F_gen, predict ,B_gen, 
                       Q_acc, Q_other):
"""


EKF_filter = EKFNet.EKFNet()
EKF_filter.load_data_set(dataset)

x_post = np.random.randn(5,1)
p_post = np.random.randn(5,5)
Q_acc   = np.random.randn(2,2)
Q_other = np.random.randn(5,5)
dt = 0.5
F_gen   = EKF_filter.G
predict = EKF_filter.predict
B_gen   = EKF_filter.gen_B_simple


x_pred, p_pred, cache = EKFNet_layers.prediction_forward(x_post, p_post, dt, F_gen, predict ,B_gen, Q_acc, Q_other)

#dx_pred =  np.random.randn(*x_pred.shape)
#dp_pred = np.zeros(p_pred.shape)

dx_pred =  np.zeros(x_pred.shape)
dp_pred =  np.random.randn(*p_pred.shape)


fx_post = lambda x_post : EKFNet_layers.prediction_forward(x_post, p_post, dt, F_gen, predict ,B_gen, Q_acc, Q_other)[1]
fp_post = lambda p_post : EKFNet_layers.prediction_forward(x_post, p_post, dt, F_gen, predict ,B_gen, Q_acc, Q_other)[1]
fQ_acc  = lambda Q_acc  : EKFNet_layers.prediction_forward(x_post, p_post, dt, F_gen, predict ,B_gen, Q_acc, Q_other)[1]
fQ_other= lambda Q_other: EKFNet_layers.prediction_forward(x_post, p_post, dt, F_gen, predict ,B_gen, Q_acc, Q_other)[1]



dx_post_num  = eval_numerical_gradient_array(fx_post, x_post, dp_pred)
dp_post_num  = eval_numerical_gradient_array(fp_post, p_post, dp_pred) 
dQ_acc_num   = eval_numerical_gradient_array(fQ_acc,  Q_acc , dp_pred)
dQ_other_num = eval_numerical_gradient_array(fQ_other,Q_other,dp_pred)

dx_post, dp_post, dQ_acc, dQ_other = EKFNet_layers.prediction_backward(dx_pred, dp_pred, cache)


print(dQ_other_num)
print(dQ_other)

print('dx_pred error: ', rel_error(dx_post_num, dx_post))
print('dp_pred error: ', rel_error(dp_post_num, dp_post))
print('dQ_acc  error: ', rel_error(dQ_acc_num , dQ_acc))
print('dQ_other error:', rel_error(dQ_other_num, dQ_other))



2.71
[[-1.3365  0.4335  0.5246 -0.2355 -0.1436]
 [ 1.6885  1.8314 -0.6116  0.0124  0.9221]
 [-0.9319 -0.9455  0.1211  0.9402  1.4823]
 [ 0.9526 -0.7521 -1.0114  0.5792 -0.6966]
 [ 0.2804  0.9102  1.0658 -0.4772  0.9948]]
[[-1.3365  0.4335  0.5246 -0.2355 -0.1436]
 [ 1.6885  1.8314 -0.6116  0.0124  0.9221]
 [-0.9319 -0.9455  0.1211  0.9402  1.4823]
 [ 0.9526 -0.7521 -1.0114  0.5792 -0.6966]
 [ 0.2804  0.9102  1.0658 -0.4772  0.9948]]
dx_pred error:  2.96184105038635e-11
dp_pred error:  8.151492009652795e-11
dQ_acc  error:  1.197103002431942e-10
dQ_other error: 7.826703837559119e-12


In [491]:
x_post = np.random.randn(5,1)
p_post = np.random.randn(5,5)
Q_acc   = np.random.randn(2,2)
Q_other = np.random.randn(5,5)
dt = 0.5
F_gen   = EKF_filter.G
predict = EKF_filter.predict
B_gen   = EKF_filter.gen_B_simple


x_pred, p_pred, cache = EKFNet_layers.prediction_forward(x_post, p_post, dt, F_gen, predict ,B_gen, Q_acc, Q_other)

dx_pred =  np.random.randn(*x_pred.shape)
dp_pred = np.zeros(p_pred.shape)

#dx_pred =  np.zeros(x_pred.shape)
#dp_pred =  np.random.randn(*p_pred.shape)


fx_post = lambda x_post : EKFNet_layers.prediction_forward(x_post, p_post, dt, F_gen, predict ,B_gen, Q_acc, Q_other)[0]
fp_post = lambda p_post : EKFNet_layers.prediction_forward(x_post, p_post, dt, F_gen, predict ,B_gen, Q_acc, Q_other)[0]
fQ_acc  = lambda Q_acc  : EKFNet_layers.prediction_forward(x_post, p_post, dt, F_gen, predict ,B_gen, Q_acc, Q_other)[0]
fQ_other= lambda Q_other: EKFNet_layers.prediction_forward(x_post, p_post, dt, F_gen, predict ,B_gen, Q_acc, Q_other)[0]



dx_post_num  = eval_numerical_gradient_array(fx_post, x_post, dx_pred)
dp_post_num  = eval_numerical_gradient_array(fp_post, p_post, dx_pred) 
dQ_acc_num   = eval_numerical_gradient_array(fQ_acc,  Q_acc , dx_pred)
dQ_other_num = eval_numerical_gradient_array(fQ_other,Q_other,dx_pred)

dx_post, dp_post, dQ_acc, dQ_other = EKFNet_layers.prediction_backward(dx_pred, dp_pred, cache)


print(dx_post)
print(dx_p)

print('dx_pred error: ', rel_error(dx_post_num, dx_post))
print('dp_pred error: ', rel_error(dp_post_num, dp_post))
print('dQ_acc  error: ', rel_error(dQ_acc_num , dQ_acc))
print('dQ_other error:', rel_error(dQ_other_num, dQ_other))


[[-1.4082]
 [-1.747 ]
 [ 0.748 ]
 [-0.8924]
 [-0.2712]]
[[-1.4082]
 [-1.747 ]
 [ 0.748 ]
 [-0.8924]
 [-0.2712]]
dx_pred error:  1.0179568524450968e-11
dp_pred error:  0.0
dQ_acc  error:  0.0
dQ_other error: 0.0


### Check the input is dx_pred

#### Check the process measurement Noise

In [433]:
x_post = np.random.randn(5)
p_post = np.random.randn(5,5)
Q_acc   = np.random.randn(2,2)
Q_other = np.random.randn(5,5)
dt = 0.5
F_gen   = EKF_filter.G
predict = EKF_filter.predict
B_gen   = EKF_filter.gen_B_simple

Q, cache = EKFNet_layers.processNoise_forward(x_post, dt, B_gen, Q_acc, Q_other)

dQ =   np.random.randn(*Q.shape)

fQ_acc   = lambda Q_acc: EKFNet_layers.processNoise_forward(x_post, dt, B_gen, Q_acc, Q_other)[0]
fQ_other = lambda Q_other: EKFNet_layers.processNoise_forward(x_post, dt, B_gen, Q_acc, Q_other)[0]
fx_post =  lambda x_post: EKFNet_layers.processNoise_forward(x_post, dt, B_gen, Q_acc, Q_other)[0]

dQ_acc_num   = eval_numerical_gradient_array(fQ_acc, Q_acc, dQ)
dQ_other_num = eval_numerical_gradient_array(fQ_other, Q_other, dQ)
dx_post_num =  eval_numerical_gradient_array(fx_post, x_post, dQ)

dQ_acc, dQ_other, dx_post = EKFNet_layers.processNoise_backward(dQ, cache)

print('dQ_acc error: ', rel_error(dQ_acc_num, dQ_acc))
print('dQ_other error : ', rel_error(dQ_other_num, dQ_other))
print('dx_post error  : ', rel_error(dx_post_num, dx_post))
print(dx_post_num)
print(dx_post)

dQ_acc error:  2.0812798649559835e-11
dQ_other error :  3.2756427840275527e-12
dx_post error  :  3.3536474820063427e-12
[0.     0.     0.0525 0.     0.    ]
[0.     0.     0.0525 0.     0.    ]
